# 017 · Advanced NGLview usage

Authors:

- First and last name, year(s) of contribution, lab, institution
- First and last name, year(s) of contribution, lab, institution

## Aim of this talktorial


    For this notebook, we should cover:

    * Simple interactive widget controls with callbacks
    * More complex controls and GUIs
    * Executing JS snippets directly for the nglview parts that are not wrapped in Python


### Contents in *Theory*

*These points should refer to the headlines of your theory section below.*

* ChEMBL database
* Compound activity measures

### Contents in *Practical*

*These points should refer to the headlines of your practical section below.*

* Connect to ChEMBL database
* Load and draw molecules

### References

* Paper 
* Tutorial links
* Other useful resources

*We suggest the following citation style:*
* Keyword describing resource: <i>Journal</i> (year), <b>volume</b>, pages (link to resource) 

*Example:*
* ChEMBL web services: <i>Nucleic Acids Res.</i> (2015), <b>43</b>, 612-620 (https://academic.oup.com/nar/article/43/W1/W612/2467881) 

## Theory

### ChEMBL database

Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.

**Note**: Include images like this (place images in `images/`):

![ChEMBL web service schema](images/chembl_webservices_schema_diagram.jpg)

*Figure 1:* 
Describe figure and add reference.
Figure and description taken from: [<i>Nucleic Acids Res.</i> (2015), <b>43</b>, 612-620](https://academic.oup.com/nar/article/43/W1/W612/2467881).

### Compound activity measures

Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.

Please use LaTeX to format formulas: $pIC_{50} = -log_{10}(IC_{50}*10^{-9})= 9-log_{10}(IC_{50}) $

If you place links, please link descriptive words.
> **Yes**:  This figure is licensed under [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/).

> **No**: This figure is licensed under CC BY-SA 3.0 ([here](https://creativecommons.org/licenses/by-sa/3.0/)).

## Practical

Short summay of what will be done in this practical section.

Please add all your imports on top of this section.

In [1]:
from pathlib import Path

Add globals to this talktorial's path (`HERE`) and its data folder (`DATA`).

In [2]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

### Connect to ChEMBL database

Explain what you will do here in the Markdown cell. This includes everything that has to do with the talktorial's storytelling.

In [3]:
# Add comments in the Code cell if you want to comment on coding decisions

In [ ]:
# ⚠ This is probably too advanced for this notebook! -> T017
#    Just use a simple visualization with thin cylinders and
#    no interactive widget controls


# JavaScript code needed to update residues around the ligand
# because this part is not exposed in the Python widget
# Based on: http://nglviewer.org/ngl/api/manual/snippets.html
_RESIDUES_AROUND = """
var protein = this.stage.compList[0];
var ligand_center = this.stage.compList[{index}].structure.atomCenter();
var around = protein.structure.getAtomSetWithinPoint(ligand_center, {radius});
var around_complete = protein.structure.getAtomSetWithinGroup(around);
var last_repr = protein.reprList[protein.reprList.length-1];
protein.removeRepresentation(last_repr);
protein.addRepresentation("licorice", {{sele: around_complete.toSeleString()}});
"""


def show_docking(protein, ligands, vina_output):
    # Split the multi PDBQT ligand file into separate files
    ligands_files = split_pdbqt(ligands)
    # Retrieve affinities (we only need that column of the dataframe)
    affinities = parse_output(vina_output)["Affinity (kcal/mol)"]

    # Create viewer widget
    viewer = create_viewer(protein, ligands_files, affinities)

    # Create selection widget
    #   Options is a list of (text, value) tuples. When we click on select, the value will be passed
    #   to the callable registered in `.observe(...)`
    selector = Select(
        options=[(f"#{i} {aff} kcal/mol", i) for (i, aff) in enumerate(affinities, 1)],
        description="",
        rows=len(ligands_files),
        layout=Layout(width="auto"),
    )

    # Arrange GUI elements
    # The selection box will be on the left, the viewer will occupy the rest of the window
    display(AppLayout(left_sidebar=selector, center=viewer, pane_widths=[1, 6, 1]))

    # This is the event handler - action taken when the user clicks on the selection box
    # We need to define it here so it can "see" the viewer variable
    def _on_selection_change(change):
        # Update only if the user clicked on a different entry
        if change["name"] == "value" and (change["new"] != change["old"]):
            viewer.hide(
                list(range(1, len(ligands_files) + 1))
            )  # Hide all ligands (components 1-n)
            component = getattr(viewer, f"component_{change['new']}")
            component.show()  # Display the selected one
            component.center(500)  # Zoom view
            # Call the JS code to show sidechains around ligand
            viewer._execute_js_code(_RESIDUES_AROUND.format(index=change["new"], radius=5))

    # Register event handler
    selector.observe(_on_selection_change)
    # Trigger event manually to focus on the first solution
    _on_selection_change({"name": "value", "new": 1, "old": None})

    return viewer

In the function above we are doing some advanced Python. If you are interested, you can click on the header below to view some more details.

<br />

<details>
    
<summary>Advanced Python explanation</summary>
    
> First you might have noticed is that we are mixing JavaScript and Python code! This is possible thanks to the `ipywidgets` infrastructure, as provided in `NGLViewer.Viewer._execute_js_code`. You can pass a string here (containing JavaScript code) and it will be executed in the NGL widget scope (`this` refers to the interactive canvas). To make it parameterizable, we have added some template placeholders that are formatted on each call to `_on_selection_change`.
>
> This function, `_on_selection_change` is the glue that ties user interactions (clicks on the selection box) to the Python world. It will be called each time you click on the selection box, with a single argument `change`: a dictionary containing the type of event and both the old and new values (so you can compare and do something about it). However, we also need `viewer` to be present in that function... and [we cannot pass additional values](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Traitlet-events)!
>
> One way to have `viewer` available in that function is to nest its definition within our `show_docking` function. That way, it will be able to access the outer scope without having to look up in the notebook (global) scope. You have some [more information about scope and closures in this post](https://medium.com/@dannymcwaves/a-python-tutorial-to-understanding-scopes-and-closures-c6a3d3ba0937).

</details>


### Load and draw molecules

Explain what you will do.

In [ ]:
color_map = {
    "hydrophobic": [0.90, 0.10, 0.29],
    "hbond": [0.26, 0.83, 0.96],
    "waterbridge": [1.00, 0.88, 0.10],
    "saltbridge": [0.67, 1.00, 0.76],
    "pistacking": [0.75, 0.94, 0.27],
    "pication": [0.27, 0.60, 0.56],
    "halogen": [0.94, 0.20, 0.90],
    "metal": [0.90, 0.75, 1.00],
}


def show_docking_and_interactions(protein, ligands, vina_output):
    # Split the multi PDBQT ligand file into separate files
    ligands_files = split_pdbqt(ligands)
    # Retrieve affinities (we only need that column of the dataframe)
    affinities = parse_output(vina_output)["Affinity (kcal/mol)"]

    # Create selection widget
    #   Options is a list of (text, value) tuples. When we click on select, the value will be passed
    #   to the callable registered in `.observe(...)`
    selector = Select(
        options=[(f"#{i} {aff} kcal/mol", i - 1) for (i, aff) in enumerate(affinities, 1)],
        description="",
        rows=len(ligands_files),
        layout=Layout(width="auto"),
    )

    # Arrange GUI elements
    # The selection box will be on the left, the viewer will occupy the rest of the window (but it will be added later)
    app = AppLayout(left_sidebar=selector, center=None, pane_widths=[1, 6, 1], height="600px",)

    # This is the event handler - action taken when the user clicks on the selection box
    # We need to define it here so it can "see" the viewer variable
    def _on_selection_change(change):
        # Update only if the user clicked on a different entry
        if change["name"] == "value" and (change["new"] != change["old"]):
            if app.center is not None:
                app.center.close()

            # NGL Viewer
            app.center = viewer = nv.NGLWidget(height="600px", default=True, gui=True)
            prot_component = viewer.add_component(protein, ext="pdbqt")  # add protein
            time.sleep(1)
            lig_component = viewer.add_component(
                ligands_files[change["new"]], ext="pdbqt"
            )  # add selected ligand
            time.sleep(1)
            lig_component.center(duration=500)

            # Add interactions
            interactions_by_site = analyze_interactions(
                f"data/docked_protein_ligand.{change['new']+1}.pdb"
            )
            interacting_residues = []
            for site, interactions in interactions_by_site.items():
                for interaction_type, interaction_list in interactions.items():
                    color = color_map[interaction_type]
                    if len(interaction_list) == 1:
                        continue
                    df_interactions = pd.DataFrame.from_records(
                        interaction_list[1:], columns=interaction_list[0]
                    )
                    for _, interaction in df_interactions.iterrows():
                        name = interaction_type
                        viewer.shape.add_cylinder(
                            interaction["LIGCOO"], interaction["PROTCOO"], color, [0.1], name,
                        )
                        interacting_residues.append(interaction["RESNR"])
            # Display interacting residues
            res_sele = " or ".join([f"({r} and not _H)" for r in interacting_residues])
            res_sele_nc = " or ".join(
                [f"({r} and ((_O) or (_N) or (_S)))" for r in interacting_residues]
            )
            prot_component.add_ball_and_stick(
                sele=res_sele, colorScheme="chainindex", aspectRatio=1.5
            )
            prot_component.add_ball_and_stick(
                sele=res_sele_nc, colorScheme="element", aspectRatio=1.5
            )

    # Register event handler
    selector.observe(_on_selection_change)
    # Trigger event manually to focus on the first solution
    _on_selection_change({"name": "value", "new": 1, "old": None})

    return app

`show_docking_and_interactions` is mostly the same function as `show_docking`, but it will also calculate the PLIP interactions for each selection event. In this case, the event handler is a bit more involved. We have to recreate the NGL viewer for each selection event because the interaction cylinders cannot be manually erased. However, this time there is no JavaScript involved; just pure Python. Recreating the viewer widget from scratch, loading the molecules and calculating the interactions takes more time, though, and as a result the experience is not that dynamic.

## Discussion

We first used event handlers (as registered by `.observe()`) to interactively update the viewer with the selected protein-ligand pose and its surroundings. Then, we extended the visualization using PLIP's automated analysis to draw the interactions in the viewer.

## Quiz

Ask three questions that the user should be able to answer after doing this talktorial. Choose important take-aways from this talktorial for your questions.

1. Question
2. Question
3. Question